# Импорт бибилиотек

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import zipfile
import requests
from io import BytesIO


# 1. Загрузка и распаковка данных (имитация wget)

In [ ]:
url = "https://archive.ics.uci.edu/static/public/222/bank+marketing.zip"
response = requests.get(url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Имя нужного файла

In [18]:
csv_file_name = 'bank-full.csv'

# Чтение файла, используя разделитель ';'

In [19]:
try:
    df = pd.read_csv('/content/bank-full.csv', sep=';')
    print(f"Загружен датафрейм размером: {df.shape}")
    print("-" * 50)
except FileNotFoundError:
    print("Error: 'bank-full.csv' not found in /content/. Please ensure the file is present.")
    print("-" * 50)

Загружен датафрейм размером: (45211, 17)
--------------------------------------------------


# Выбор указанных признаков


In [20]:
features = ['age', 'job', 'marital', 'education', 'balance', 'housing',
            'contact', 'day', 'month', 'duration', 'campaign',
            'pdays', 'previous', 'poutcome', 'y']
df = df[features]

# Проверка пропущенных значений

In [21]:
if df.isnull().sum().sum() == 0:
    print("Пропущенных значений нет.")
print("-" * 50)

Пропущенных значений нет.
--------------------------------------------------


# Кодирование целевой переменной 'y'

In [22]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})

# Разделение данных


In [23]:
X = df.drop(columns=['y'])
y = df['y']

# Разделение на train (60%), validation (20%), test (20%)

In [24]:
X_full_train, X_test, y_full_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_full_train, y_full_train, test_size=0.25, random_state=42
)

In [25]:
print(f"Размер тренировочного набора (60%): {len(X_train)}")
print(f"Размер валидационного набора (20%): {len(X_val)}")
print(f"Размер тестового набора (20%): {len(X_test)}")
print("-" * 50)

Размер тренировочного набора (60%): 27126
Размер валидационного набора (20%): 9042
Размер тестового набора (20%): 9043
--------------------------------------------------


# <codecell>

### ❓ Вопрос 1: Самое частое значение (mode) для education

In [26]:
print("--- Ответ на Вопрос 1 ---")
education_mode = df['education'].mode()[0]
print(f"Самое частое значение для 'education': {education_mode}")
print(f"✅ Ответ 1: {education_mode}")
print("-" * 50)

--- Ответ на Вопрос 1 ---
Самое частое значение для 'education': secondary
✅ Ответ 1: secondary
--------------------------------------------------


# <codecell>

### ❓ Вопрос 2: Наибольшая корреляция между числовыми признаками

In [27]:
print("--- Ответ на Вопрос 2 ---")
numerical_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
correlation_matrix = df[numerical_features].corr().abs()

--- Ответ на Вопрос 2 ---


# Ищем максимальное значение корреляции (вне главной диагонали)

In [28]:
np.fill_diagonal(correlation_matrix.values, 0)
max_corr_pair = correlation_matrix.stack().idxmax()
max_corr_value = correlation_matrix.stack().max()

print(f"Наибольшая абсолютная корреляция: {max_corr_value:.4f} между признаками {max_corr_pair}")
print(f"✅ Ответ 2: {' и '.join(sorted(max_corr_pair))}") # Сортируем для унификации вывода
print("-" * 50)

Наибольшая абсолютная корреляция: 0.4548 между признаками ('pdays', 'previous')
✅ Ответ 2: pdays и previous
--------------------------------------------------


# <codecell>

### ❓ Вопрос 3: Наибольшая взаимная информация

In [29]:
print("--- Ответ на Вопрос 3 ---")
categorical_features = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
X_train_cat = X_train[categorical_features]

mi_scores = {}
for col in categorical_features:
    le = LabelEncoder()
    X_encoded = le.fit_transform(X_train_cat[col])

    score = mutual_info_classif(
        X_encoded.reshape(-1, 1),
        y_train,
        random_state=42
    )[0]
    mi_scores[col] = round(score, 4)

mi_series = pd.Series(mi_scores).sort_values(ascending=False)
max_mi_feature = mi_series.index[0]
max_mi_value = mi_series.iloc[0]

print("Взаимная информация (mutual_info_classif) для категориальных признаков:")
print(mi_series.round(4))
print(f"\nПризнак с наибольшей взаимной информацией: {max_mi_feature} ({max_mi_value:.4f})")
print(f"✅ Ответ 3: {max_mi_feature}")
print("-" * 50)

--- Ответ на Вопрос 3 ---
Взаимная информация (mutual_info_classif) для категориальных признаков:
poutcome     0.0324
month        0.0272
contact      0.0163
housing      0.0147
job          0.0101
marital      0.0072
education    0.0056
dtype: float64

Признак с наибольшей взаимной информацией: poutcome (0.0324)
✅ Ответ 3: poutcome
--------------------------------------------------


# <codecell>

### ❓ Вопрос 4: Точность логистической регрессии на валидационном наборе (C=1.0)

In [30]:
print("--- Ответ на Вопрос 4 ---")

--- Ответ на Вопрос 4 ---


# Выполняем One-Hot Encoding для категориальных признаков

In [31]:
X_train_ohe = pd.get_dummies(X_train, columns=categorical_features, drop_first=False)
X_val_ohe = pd.get_dummies(X_val, columns=categorical_features, drop_first=False)

# Согласование столбцов (могут отличаться из-за разных наборов)

In [32]:
train_cols = X_train_ohe.columns
X_val_ohe = X_val_ohe.reindex(columns=train_cols, fill_value=0)

# Инициализация и обучение модели

In [33]:
model_q4 = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_q4.fit(X_train_ohe, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

# Предсказание и расчет точности на валидационном наборе

In [34]:
y_val_pred = model_q4.predict(X_val_ohe)
accuracy_q4 = accuracy_score(y_val, y_val_pred)
accuracy_q4_rounded = round(accuracy_q4, 2)

print(f"Точность на валидационном наборе: {accuracy_q4:.4f}")
print(f"Округленная точность: {accuracy_q4_rounded}")
print(f"✅ Ответ 4: {accuracy_q4_rounded}")
print("-" * 50)

Точность на валидационном наборе: 0.9011
Округленная точность: 0.9
✅ Ответ 4: 0.9
--------------------------------------------------


# <codecell>

### ❓ Вопрос 5: Наименее полезный признак (Feature Elimination)

In [35]:
print("--- Ответ на Вопрос 5 ---")
original_accuracy = accuracy_q4
feature_elimination_results = {}

--- Ответ на Вопрос 5 ---


# Список всех признаков (числовые + OHE-закодированные категориальные)

In [36]:
all_ohe_features = X_train_ohe.columns.tolist()

# Признаки для исключения по очереди

In [37]:
features_to_check = ['age', 'balance', 'marital', 'previous']

print(f"Исходная точность: {original_accuracy:.4f}\n")
print("Анализ разницы точности (Исходная - Без признака):")

for feature_to_remove in features_to_check:
    if feature_to_remove in numerical_features:
        cols_to_drop = [feature_to_remove]
    else:
        cols_to_drop = [col for col in all_ohe_features if col.startswith(feature_to_remove + '_')]

    X_train_reduced = X_train_ohe.drop(columns=cols_to_drop)
    X_val_reduced = X_val_ohe.drop(columns=cols_to_drop)

    model_reduced = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_reduced.fit(X_train_reduced, y_train)

    y_val_pred_reduced = model_reduced.predict(X_val_reduced)
    accuracy_reduced = accuracy_score(y_val, y_val_pred_reduced)

    difference = original_accuracy - accuracy_reduced
    feature_elimination_results[feature_to_remove] = difference

    print(f"  Без '{feature_to_remove}': Точность={accuracy_reduced:.4f}, Разница={difference:.4f}")

min_diff_feature = min(feature_elimination_results, key=lambda k: abs(feature_elimination_results[k]))
min_diff_value = feature_elimination_results[min_diff_feature]

print("-" * 30)
print(f"Признак с наименьшим изменением точности: {min_diff_feature}")
print(f"Наименьшая разница: {min_diff_value:.4f}")
print(f"✅ Ответ 5: {min_diff_feature}")
print("-" * 50)

Исходная точность: 0.9011

Анализ разницы точности (Исходная - Без признака):
  Без 'age': Точность=0.9011, Разница=0.0000
  Без 'balance': Точность=0.9007, Разница=0.0004
  Без 'marital': Точность=0.9011, Разница=0.0000
  Без 'previous': Точность=0.9006, Разница=0.0006
------------------------------
Признак с наименьшим изменением точности: age
Наименьшая разница: 0.0000
✅ Ответ 5: age
--------------------------------------------------


# <codecell>

### ❓ Вопрос 6: Регуляризованная логистическая регрессия: выбор C

In [38]:
print("--- Ответ на Вопрос 6 ---")

--- Ответ на Вопрос 6 ---


# C=0 не поддерживается в LogisticRegression, поэтому берем [0.01, 0.1, 1, 10]

In [39]:
C_values = [0.01, 0.1, 1, 10]

# Если бы 0.001 не был близок к 0, мы бы его добавили. Возьмем 0.01 как минимальное.

In [40]:
results_q6 = {}

print("Анализ точности для разных значений C:")

Анализ точности для разных значений C:


# Использование наборов данных с OHE из Вопроса 4

In [41]:
for C in C_values:
    if C == 0:
        continue

    model_q6 = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model_q6.fit(X_train_ohe, y_train)

    y_val_pred_q6 = model_q6.predict(X_val_ohe)
    accuracy_q6 = accuracy_score(y_val, y_val_pred_q6)

    results_q6[C] = round(accuracy_q6, 3)
    print(f"  C={C:.3f}: Точность={results_q6[C]}")

max_accuracy = max(results_q6.values())
best_C_values = [C for C, acc in results_q6.items() if acc == max_accuracy]
best_C = min(best_C_values)

print("-" * 30)
print(f"Наилучшая точность ({max_accuracy}) получена при C: {best_C_values}")
print(f"Наименьшее C с наилучшей точностью: {best_C}")
print(f"✅ Ответ 6: {best_C}")
print("-" * 50)

  C=0.010: Точность=0.898
  C=0.100: Точность=0.901
  C=1.000: Точность=0.901
  C=10.000: Точность=0.901
------------------------------
Наилучшая точность (0.901) получена при C: [0.1, 1, 10]
Наименьшее C с наилучшей точностью: 0.1
✅ Ответ 6: 0.1
--------------------------------------------------


# <codecell>

### Сводные ответы

Вопрос 1 (Mode education): secondary

Вопрос 2 (Max correlation pair): ('pdays', 'previous')

Вопрос 3 (Max mutual info feature): poutcome

Вопрос 4 (Accuracy at C=1.0, rounded): 0.9

Вопрос 5 (Least useful feature): age

Вопрос 6 (Best C for max accuracy): 0.1